Car Pipeline:
Car Detection (YOLO) -> detect and classify driver and seatbelt (Custom) Not Done Yet -> license plate detection(YOLO) -> OCR (pytesseract)


In [ ]:
import sys
import os 
if 'google.colab' in sys.modules:
    !pip -q install ultralytics
    !pip -q install pytesseract
    !apt-get -qq install tesseract-ocr
    if not os.path.exists('license-plate-finetune-v1s.pt'):
        !wget https://huggingface.co/morsetechlab/yolov11-license-plate-detection/resolve/main/license-plate-finetune-v1s.pt


In [ ]:
from ultralytics import YOLO
from ultralytics.engine.results import Results
from pathlib import Path
import pytesseract
from PIL import Image
import numpy as np
from typing import Any

license_plate_detection_model = YOLO('license-plate-finetune-v1s.pt')

car_detection_model = YOLO("yolo11n.pt")


In [ ]:
def detect_cars(image_path: Path | str) -> list[np.ndarray]:
    # First, detect cars in the image
    car_results = car_detection_model.predict(source=image_path, classes=[2,5,7], verbose=False)  # COCO classes for car, bus, truck
    output = []
    for car in car_results:
        for box in car.boxes:
            # Extract car bounding box
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            car_image = car_results[0].orig_img[y1:y2, x1:x2]
            output.append(car_image)  
    return output

In [ ]:
def plate_detection(car_images: list[np.ndarray]) -> list[Results]:
    plate_results = license_plate_detection_model.predict(source=car_images, verbose=False)
    return plate_results

In [ ]:
def get_plate_images(plate_results: list[Results]) -> list[np.ndarray]:
    output = []
    for plate in plate_results:
        for box in plate.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            plate_image = plate.orig_img[y1:y2, x1:x2]
            output.append(plate_image)
    return output   

In [ ]:
def run_ocr_on_plate(plate_images: list[np.ndarray])->dict[str, np.ndarray]:
    ocr_results = {}
    for plate in plate_images:
        
        # Convert numpy array to PIL Image for pytesseract
        pil_image = Image.fromarray(plate)
        ocr_result = pytesseract.image_to_string(pil_image, config='--psm 7')
        ocr_results[ocr_result.strip()] = plate
    return ocr_results


In [ ]:
def run_pipeline(image_path: Path | str):
    car_images = detect_cars(image_path)
    plate_results = plate_detection(car_images)
    plate_images = get_plate_images(plate_results)
    ocr_results = run_ocr_on_plate(plate_images)

    return ocr_results

In [ ]:
if 'google.colab' in sys.modules:
    dirl = os.listdir(".")
    if "bmwalleng.png" not in dirl:
      !wget https://github.com/AFAskar/dlimageclass/blob/main/bmwalleng.png?raw=true
      !mv bmwalleng.png?raw=true bmwalleng.png
    if '2cars.png' not in dirl:
      !wget https://github.com/AFAskar/dlimageclass/blob/main/2cars.png?raw=true
      !mv 2cars.png?raw=true 2cars.png


In [ ]:
def output(images_path: Path | str,output_dir:Path|str="output") -> dict[str, str | list[str | Path]]:
    if isinstance(images_path, str):
        images_path=Path(images_path)
    images_path.mkdir(parents=True, exist_ok=True)
    if isinstance(output_dir, str):
        output_dir=Path(output_dir)
        
    output_dir.mkdir(parents=True, exist_ok=True)
    
    ocr_results = run_pipeline(images_path)
    image_paths = []
    output = {}
    # save images with detected plates
    for plate_text, plate_image in ocr_results.items():
        safe_plate_text = plate_text.replace(" ", "_").replace("\n", "")
        image_path = output_dir / f"{safe_plate_text}.png"
        Image.fromarray(plate_image).save(image_path)
        image_paths.append(str(image_path))
        output[plate_text] = str(image_path)

    json_output=output_dir / "output.json"
    with open(json_output, "w") as f:
        json.dump(output, f, indent=4)
    
    
    return output

In [ ]:
import json

images = ['./bmwalleng.png','./2cars.png']
all_results = []
for img_path in images:
    plate_images, plate_results = run_pipeline(img_path)
    ocr_results = run_ocr_on_plate(plate_images)    
    all_results.extend(plate_results)

# Only show results that have detected plates
for plate in all_results:
    if len(plate.boxes) > 0:  # Only show if there are detections
        plate.show()
        
with open('ocr_results.json', 'w') as f:
    json.dump(ocr_results, f)